# Exercice 3 - Merry Christmhash
⚠️ Run this with an ipython kernel, not sage

In [1]:
import binascii
import struct
import operator
import random
import math

## Parameters

In [2]:
# avoids undesired parameters printing when commented
dummy_variable=1

### Tom

In [3]:

Q3a_M="If you are crying, fight against it! If you're regretting, walk forward! Only complaining on your misfortune, you're nothing but a common pig! (Ciel Phantomhive, Kuroshitsuji)"
Q3a_H=[1712935222, 2277794147, 2376157840, 3916784640, 1346939745, 2708679518, 85473800, 3788905359]
Q3a_K=[1943621227, 886196854, 2945353417, 208223340, 1212883955, 2473679821, 3054683082, 2029524489, 3963858188, 3339906788, 2823505815, 3350944332, 312194943, 3986347121, 2144670729, 741633462, 3663151569, 3544144315, 2690702324, 1848842070, 811599001, 510224092, 4281468752, 752009382, 1104123092, 3497858695, 2695667604, 1727413627, 3050579566, 510494887, 4285911224, 952491461, 4237080290, 10651148, 4015126669, 154644693, 1282241408, 2068274028, 2010214286, 3416638395, 2453720085, 3229980118, 2094058516, 1197555596, 922344893, 2154715712, 3908004108, 481673144, 1957904962, 3050061601, 541865778, 1006059256, 585392047, 1178853390, 2805354064, 972068900, 3754759061, 1435964078, 398864381, 2460187893, 1385904313, 3634606251, 2462881886, 3619520975]
Q3a_S=[[22, 16, 27], [21, 4, 23]]
Q3b_t="011e9faabb4e5381"
Q3b_d="384bba4b1dbd84001ccb55791cfcb7ac4da9f6f67174102751830195e3ca5fa6"
Q3b_H=[1410008662, 3165996059, 4265820587, 3228924364, 2876772144, 502571215, 1296440626, 3528054622]
Q3b_K2=[1660732939, 2978614430, 479083038, 4039110515, 503760146, 2160178725, 3371169074, 3023444970, 1832019271, 1110792947, 4275626369, 1675250051, 2784406696, 4257265674, 517985909, 3086983861, 3409296999, 1504386796, 374146206, 4055850561, 2556892257, 2255194630, 2066495719, 259846162, 593904479, 2868322264, 3146981800, 3913516318, 625630895, 1491933359, 1856807117, 3795630429, 3035508049, 3725788811, 2587675021, 4081590610, 1694559383, 2421525974, 2954023209, 3274021365, 556994319, 3840950079, 2748231069, 4293925912, 1934082025, 683379336, 3605902893, 3811110963, 3312404527, 562459431, 3079990482, 4161695928, 1789332738, 3133889965, 94188891, 1773848693]
Q3b_S=[[8, 1, 2], [9, 5, 13]]

dummy_variable=1

### Manon

In [27]:
Q3a_M="She beat me like I owed her money. (Kurosaki Ichigo, Bleach)"
Q3a_H=[2058406342, 2402872068, 2655990099, 2555730242, 2099307039, 1755366654, 1315484406, 2266643364]
Q3a_K=[3161650145, 2478786008, 3596976753, 23365742, 2902372063, 3140174779, 1685917262, 4095701929, 1735303610, 4029073857, 2156448346, 2747723612, 425230121, 4195693004, 2262359291, 2729153660, 4092419666, 945787262, 3783309411, 2202091746, 64310674, 1007531533, 610686592, 2456381975, 3394979241, 876311637, 1808390050, 2056668524, 433042482, 2408908162, 1117892050, 428200906, 44952149, 1140450444, 4170832219, 2295799987, 1651692986, 2264348356, 3167976031, 1162439454, 229359305, 1680500478, 38520716, 1247652517, 3569222990, 2204012769, 1187255224, 2880492565, 1590741071, 1840791506, 1602749606, 269435894, 104709090, 1862465305, 1289366085, 3565398753, 2255931833, 1243912702, 4039233372, 1401482742, 3460753915, 307271959, 3171498041, 3294578001]
Q3a_S=[[8, 7, 3], [23, 14, 17]]
Q3b_t="ee727991d3afb324"
Q3b_d="0f2fb2bf1c1cf18c01a843d92481f28f21bebd611d9505d6796fbd4b29d4537d"
Q3b_H=[2981129015, 3780026285, 2104308789, 3557758737, 1127036048, 495180949, 156842781, 375376189]
Q3b_K2=[2258015820, 3543266183, 1407439840, 2660815954, 4118130345, 2713191677, 91016618, 4131412193, 935349841, 1248693912, 4034271131, 967344220, 275899820, 1843097767, 1090452896, 1723149751, 2950777158, 1106007055, 2588207278, 3801494776, 1383402648, 2730830500, 3860301251, 3938887455, 947234393, 2998878898, 1732253144, 4286648010, 2567922503, 1825686364, 4238295495, 3267943707, 3739650532, 1497942724, 1623911534, 1020400754, 1783636689, 641826600, 2349031891, 350585123, 2470014875, 745862463, 3868264630, 1912944067, 2225798570, 1815044244, 621425367, 3573854289, 4087086267, 757683534, 3345295414, 2539804101, 2267403113, 3828233624, 4056885473, 1977041357]
Q3b_S=[[13, 13, 7], [8, 13, 15]]

dummy_variable=1

## $H(m,H,K,S)$ implementation

### Implementation

In [28]:
# https://200ok.ch/posts/2018-12-09_unhexlify.html
def xxd(s):
    return binascii.hexlify(s).decode()

In [29]:
def unxxd(s):
    return binascii.unhexlify(s)

In [30]:
def pack(k,n):
    if k==32:
        return struct.pack('>L',n)
    elif k==64:
        return struct.pack('>Q',n)
    else:
        print("Error, k should be 32 or 64, here it's {}".format(k))
        return None

In [31]:
assert list(pack(32,1))==[0,0,0,1]
assert list(pack(64,1))==[0,0,0,0,0,0,0,1]

In [32]:
# we must make sure ^ and ~ operators are well defined as used later
for i in range(4096):
    r1 = random.randint(0,2**64)
    r2 = random.randint(0,2**64)
    assert r1^r2 == operator.xor(r1,r2)
    assert ~r1 == operator.invert(r1)

In [33]:
def extend(M):
    mis = list(chunks(bytes(M),4))
    W = [0 for i in range(64)]
    for i in range(0,16):
        W[i] = mis[i][0]<<24|mis[i][1]<<16|mis[i][2]<<8|mis[i][3]
    for i in range(16,64):
        t1 = ((((W[i-15])>>7) | ((W[i-15])<<25)) ^ (((W[i-15])>>18) | ((W[i-15])<<14)) ^ (((W[i-15])>>3)))%2**32
        t2 = ((((W[i-2])>>17) | ((W[i-2])<<15)) ^ (((W[i-2])>>19) | ((W[i-2])<<13)) ^ (((W[i-2])>>10)))%2**32
        W[i] = (W[i-16]+W[i-7]+t1+t2)%2**32
    return W

In [34]:
def compress(x,W,K,S):
    y = x
    for i in range(64):
        s0=0
        for j in range(0,3):
            # we make all our ops modulo 2^32 as y elements are expected to be
            s0 = (s0 ^ ((y[0]>>(S[0][j])) | (y[0]<<(32-S[0][j]))))%2**32
        s1=0
        for j in range(0,3):
            s1 = (s1 ^ ((y[4]>>(S[1][j])) | (y[4]<<(32-S[1][j]))))%2**32
        t1 = (y[4] & y[5] ^ (~y[4]) & y[6])%2**32
        t2 = (s1 + t1 + y[7] + K[i] + W[i])%2**32
        t3 = (y[0] & y[1] ^ y[0] & y[2] ^ y[1] & y[2])%2**32
        
        tmp0 = (s0+t2+t3)%2**32
        tmp4 = (y[3]+t2)%2**32
        y = [tmp0,y[0],y[1],y[2],tmp4,y[4],y[5],y[6]]
    return y

In [35]:
def chunks(lis, n):
    """Yield successive n-sized chunks from list."""
    for i in range(0, len(lis), n):
        yield lis[i:i + n]

In [36]:
def H(m,H,K,S):
    m_numberofbytes = len(m)
    l = (-m_numberofbytes-9)%64
    m = bytes(m) + b'\x80'  + b'\x00' *l + pack(64,8*m_numberofbytes)
    M = list(chunks(m,64))
    d = H
    L = len(M)
    for i in range(L):
        W = extend(M[i])
        y = compress(d,W,K,S)
        d = [(di + yi) % 2**32 for di,yi in zip(d,y)] 
    d_reduced = d[:8]
    d_out = []
    for di in d_reduced:
         d_out += pack(32,di)
    return d_out

### Execution

In [37]:
digest = H(Q3a_M.encode(), Q3a_H, Q3a_K, Q3a_S)
print(digest,)

[83, 224, 193, 197, 181, 207, 191, 162, 185, 75, 250, 140, 238, 174, 34, 159, 75, 23, 203, 3, 83, 229, 11, 155, 110, 188, 109, 52, 248, 120, 8, 109]


In [38]:
Q3a_d = xxd(bytes(digest))
Q3a_d

'53e0c1c5b5cfbfa2b94bfa8ceeae229f4b17cb0353e50b9b6ebc6d34f878086d'

## Key recovery

### Invert Compression

The compression step goes as follows:

```
y = x
[computation]
return  (tmp_0, y_0, y_1, y_2, tmp_4, y_4, y_5, y_6)
```

So we need to recover $y_3$ and $y_7$.

We know that :
$tmp_0 = (s_0+t_2+t_3)\ mod\ 2^{32}
\\tmp_4 = (y_3+t_2)\ mod\ 2^{32}$

So we can find $y_3$: 
$t_2 = (tmp_0 - (s_0 + t_3))\ mod\ 2^{32}
\\y_3 = (tmp_4-t_2)\ mod\ 2^{32}$

And finally $y_7$:
$t_2 = (s_1 + t_1 + y_7 + k_i + w_i)\ mod\ 2^{32}
\\t_2 = (tmp_0 - (s_0 + t_3)) \ mod\ 2^{32}
\\y_7 = (t_2 - (k_i + s_1 + t_1 + w_i))\ mod\ 2^{32} $

All of these values are based on parameters we know, expect for recovering $y_7$ which requires us to know $k_i$ (which is fine for the upper rounds but an issue for the first 8 $k_i$ values which we are trying to uncover). 

Let's first invert the upper rounds:

In [39]:
def invert_compression_step(y,wi,ki,S):
    tmp0,y0,y1,y2,tmp4,y4,y5,y6 = y
    
    t3 = (y0 & y1 ^ y0 & y2 ^ y1 & y2)%2**32
    
    s0=0
    for j in range(0,3):
        # we make all our ops modulo 2^32 as y elements are expected to be
        s0 = (s0 ^ ((y0>>(S[0][j])) | (y0<<(32-S[0][j]))))%2**32
    s1=0
    for j in range(0,3):
        s1 = (s1 ^ ((y4>>(S[1][j])) | (y4<<(32-S[1][j]))))%2**32
        
    t2 = (tmp0 - (s0 + t3))%2**32
    
    y3 = (tmp4 - t2) %2**32
    
    t1 = (y4 & y5 ^ (~y4) & y6)%2**32
    
    y7 = (t2 - (ki + s1 + t1 + wi))%2**32
           
    og_y = y0, y1, y2, y3, y4, y5, y6, y7
    
    #print(og_y)
          
    return og_y

### Recover keys 

Since the only thing that depends on the keys is $y_7$ in the compression, we can use this to recover the keys. 

Namely, $y_7 = (t_2 - (k_i + s_1 + t_1 + w_i))\ mod\ 2^{32} $ so if we set $k_i$ to $0$ we obtain $y_7' = (t_2 - (0 + s_1 + t_1 + w_i))\ mod\ 2^{32}$. Now if we compare $y_7'$ to $y_7$ we obtain $k_i$.

In [54]:
def recover_keys(digest, t, K2, H, S):
    chunksofdigest = list(chunks(digest,8))
    digest_as_int_array = [int.from_bytes(binascii.unhexlify(d), "big") for d in chunksofdigest]
    print(digest_as_int_array)
    
    raw_state = [(d - y)  %2**32 for d, y in zip(digest_as_int_array, H)]
    
    # Padding
    t_numberofbytes = len(t)
    l = (-t_numberofbytes-9)%64
    t = bytes(t) + b'\x80'  + b'\x00' *l + pack(64,8*t_numberofbytes)
    w = extend(t)
    
    n_secret_keys = 8
    secret_keys = []
    state = raw_state
    known_keys = K2
    
    # Invert compression for known round keys (upper part)
    for i in range(1, 65 - n_secret_keys):
        state = invert_compression_step(state, w[-i], known_keys[-i], S)

    # Recover secret keys using compression algorithm with ki=0 
    f_state = state
    for i in range(7, -1, -1):
        for j in range(i + 1):
            state = invert_compression_step(state, 0, w[i - j], S)
        real_key = state[7 - i] - H[7 - i]
        secret_keys.append(real_key % 2**32)

        state = f_state
        for idx, k in enumerate(secret_keys):
            state = invert_compression_step(state, k, w[7 - idx], S)
    return secret_keys[::-1]

### Execution

In [55]:
Q3b_K1 = recover_keys(Q3b_d.encode(),unxxd(Q3b_t),Q3b_K2, Q3b_H,Q3b_S)

[254784191, 471658892, 27804633, 612496015, 566148449, 496305622, 2037366091, 701780861]


In [56]:
Q3b_K1

[2039350652,
 114788218,
 3502727220,
 4276241800,
 168065086,
 2544142094,
 265932013,
 2716758306]

### Test with Part 1 

If we correctly recover the first 8 keys, we can check our results by concatenating them with $Q3b\_K2$ and computing the digest of $Q3b\_t$ with matrix $Q3b\_S$ and initial state $Q3b\_H$.

In [57]:
AllKeys = Q3b_K1 + Q3b_K2
len(AllKeys)

64

In [58]:
recoverdigest = H(unxxd(Q3b_t), Q3b_H, AllKeys, Q3b_S)
print(recoverdigest,)

[15, 47, 178, 191, 28, 28, 241, 140, 1, 168, 67, 217, 36, 129, 242, 143, 33, 190, 189, 97, 29, 149, 5, 214, 121, 111, 189, 75, 41, 212, 83, 125]


In [59]:
should_be_d = binascii.hexlify(bytes(recoverdigest)).decode()
should_be_d

'0f2fb2bf1c1cf18c01a843d92481f28f21bebd611d9505d6796fbd4b29d4537d'

In [53]:
assert(should_be_d == Q3b_d)